In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import numpy as np

In [14]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [15]:
# X = X.drop('sub_area', axis=1)
# X_test = X_test.drop('sub_area', axis=1)
# used.append('Removed sub_area')

In [16]:
# X = pd.get_dummies(X)
# X_test = pd.get_dummies(X_test) 
# used.append('OneHot Encoding')

In [17]:
from sklearn.calibration import LabelEncoder

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
print("Train: Categorical columns:", categorical_columns)

label_encoder = LabelEncoder()

for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
print("Test: Categorical columns:", categorical_columns_test)

label_encoder = LabelEncoder()

for column in categorical_columns_test:
    X_test[column] = label_encoder.fit_transform(X_test[column])

used.append('Label Encoding')

Train: Categorical columns: ['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']
Test: Categorical columns: ['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


In [18]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [19]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [20]:
X = np.log1p(X)
X_test = np.log1p(X_test)
used.append('log Normalization')

In [21]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

<h1>With Keras<h1>

In [23]:
# features_used = X_train.shape[1]

# # optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# # optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# # Build the neural network
# os.environ["KERAS_BACKEND"] = "torch"
# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))  # You can use either dropout or early stopping
# model.add(Dense(68, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))
# model.add(Dense(40, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# # Compile the model
# model.compile(loss='mean_squared_error', optimizer=optimizerUsing)

# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

# used.append("Keras With Torch Backend")
# # used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# # used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
# used.append("3 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 64 neurons relu activation L2 regularization, Hidden Layer 3 with 40 neurons relu activation L2 regularization")
# used.append("Output linear")
# used.append("Optimizer: Adam(lr=0.01)")
# used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
# # used.append("batch_size=1800, epochs=100, early_stopping")

# # Make predictions
# predictions = model.predict(X_test)

<h1>With Skilearn<h1>

In [24]:
features_used = X_train.shape[1]

# # optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# # optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# Build the neural network using MLPRegressor from scikit-learn
model = MLPRegressor(hidden_layer_sizes=(128,64,40), activation='relu', solver='adam', shuffle=True, max_iter=100,
                     batch_size=1800, early_stopping=True, verbose=True, random_state=42)

# Train the model
model.fit(X_train, y_train)

used.append("MLPRegressor(hidden_layer_sizes=(128,64,40), activation='relu', solver='adam', shuffle=True, max_iter=100, batch_size=1800, early_stopping=True, verbose=True, random_state=42)")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
used.append("3 Hidden Layers: Hidden Layer 1 with 128 neurons, Hidden Layer 2 with 64 neurons, Hidden Layer 3 with 40 neurons")
# used.append("Output linear")
used.append("Optimizer: Adam")
# used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
used.append("batch_size=1800, epochs=100, early_stopping")

# Make predictions
predictions = model.predict(X_test)

Iteration 1, loss = 347809336745262.18750000
Validation score: -0.452463
Iteration 2, loss = 347571358890423.43750000
Validation score: -0.449599
Iteration 3, loss = 345612762750481.43750000
Validation score: -0.433104
Iteration 4, loss = 338105629505757.37500000
Validation score: -0.381921
Iteration 5, loss = 319151587610581.00000000
Validation score: -0.269652
Iteration 6, loss = 283159437970366.87500000
Validation score: -0.080040
Iteration 7, loss = 229908756366438.37500000
Validation score: 0.167306
Iteration 8, loss = 169986187922563.43750000
Validation score: 0.402627
Iteration 9, loss = 122511494927217.17187500
Validation score: 0.549031
Iteration 10, loss = 99239372625398.06250000
Validation score: 0.598737
Iteration 11, loss = 92925296001839.73437500
Validation score: 0.608128
Iteration 12, loss = 91755692837676.34375000
Validation score: 0.610364
Iteration 13, loss = 91322222547419.15625000
Validation score: 0.611913
Iteration 14, loss = 90975598177520.43750000
Validation sc

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [25]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day8.1.csv', index=False)

In [26]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 271

Label Encoding
Converted All Columns To float32
log Normalization
StandardScaler
MLPRegressor(hidden_layer_sizes=(128,64,40), activation='relu', solver='adam', shuffle=True, max_iter=100, batch_size=1800, early_stopping=True, verbose=True, random_state=42)
3 Hidden Layers: Hidden Layer 1 with 128 neurons, Hidden Layer 2 with 64 neurons, Hidden Layer 3 with 40 neurons
Optimizer: Adam
batch_size=1800, epochs=100, early_stopping
